In [ ]:
"""
This notebook is designed to showcase how to build your own subnet, modeled after the 
Bittensor Text Prompting subnet. This subnet is designed to take in a list of messages
and a list of roles, and generate a completion based on the current state of the network.

To create a subnet, you must define three (3) main elements: 
- the protocol message format (synapse subclass)
- the miner (how to complete requests)
- the validator (how to validate requests)

Below is a simple implementation of the protocol, so you can see how it works.

You can also use the `deserialize` method to specify how to deserialize the data
into a tensor. This is useful for converting strings into tensors, for example.

You must also define your priority and blacklist functions. See below:
"""


import bittensor as bt
bt.trace()

import pydantic
import time
import torch
from typing import List, Tuple, Optional, Any
from prompting.protocol import Prompting as Prompting


def prompt( synapse: Prompting ) -> Prompting:
    """
    Process the provided synapse to generate a completion.

    Args:
        synapse (Prompting): The input synapse to be processed.

    Returns:
        Prompting: The updated synapse with a completion.
    """
    bt.logging.debug("In prompt!")
    synapse.completion = "I am a chatbot"
    return synapse

def blacklist( synapse: Prompting ) -> Tuple[bool, str]:
    """
    Determines if the provided synapse should be blacklisted.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        Tuple[bool, str]: A tuple containing a boolean that indicates whether the synapse is blacklisted,
                          and a string providing the reason.
    """
    return False, ""

def priority( synapse: Prompting ) -> float:
    """
    Determines the priority of the provided synapse.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        float: The priority value of the synapse, with higher values indicating higher priority.
    """
    return 0.0


In [ ]:
# Create an Axon instance on port 8549.
axon = bt.axon(port=8559)

# Attach the forward, blacklist, and priority functions to the Axon.
# forward_fn, blacklist_fn, priority_fn: Functions for forwarding logic, blacklisting, and setting priority.
# prompt, blacklist, priority: These functions should be defined somewhere in your code.
axon.attach(
    forward_fn=prompt,
    blacklist_fn=blacklist,
    priority_fn=priority,
)

# Create a wallet instance that must be registered on the network
wallet = bt.wallet(name="miner", hotkey="default")

# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite(wallet=wallet)

try:
    # Start the Axon to begin listening for requests.
    axon.start()

    # Send a request to the Axon using the Dendrite.
    # Note: This part of the code should ideally be in an async function since 'await' is used.
    resp = await d(
        [axon],
        bt.Prompting(roles=["user"], messages=["hello, who are you?"]),
        deserialize=False
    )

    # Handle the response
    print(resp)

except Exception as e:
    print(f"An error occurred: {e}")
    
finally:
    # Ensure the Axon is properly stopped when an exception occurs or the process ends
    axon.stop()
    print("Axon stopped successfully.")
